In [2]:
import spacy
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from collections import defaultdict
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_lg')

for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True


filename = r'../raw_data/southwest_2016.txt' # r'1994_test.txt
with open(filename, 'rb') as f:
    data = [line.decode('utf-8') for line in f.readlines()]
data = ' '.join(data).replace('\r',' ').replace('/s',' ').replace('\n',' ').strip()

doc = nlp(data)

### SpaCy - Text Extraction

In [97]:
def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start : end] # matched span
    entity = span.lemma_.lower()
    sent = span.sent
        
    # add the first entity and return as there is no last match to check    
    if i == 0:
        ent_count[entity] += 1
        ent_sents[entity].append(sent)
        matched_sents.append(sent)
        return

    # get the last match to avoid adding duplicates when one term is contained
    # in another (e.g. 'integration services' in 'system integration services')
    last_match_id, last_start, last_end = matches[i-1]
    last_span = doc[last_start : last_end]
    last_entity = last_span.text.lower()
    last_sent = last_span.sent

    distinct_entity = (entity not in last_entity) or (sent != last_sent)
    not_duplicate_entity = (entity != last_entity) or (sent != last_sent)
    
    if distinct_entity and not_duplicate_entity:
        ent_count[entity] += 1
        ent_sents[entity].append(sent)
        matched_sents.append(sent)

In [98]:
matched_sents = []
ent_sents = defaultdict(list)
ent_count = defaultdict(int)
pattern = [{'POS': 'NOUN', 'OP': '+'},{'LOWER': 'services'}]
matcher = Matcher(nlp.vocab)
matcher.add('ServiceTypes', collect_sents, pattern) # add pattern
matches = matcher(doc) # match on your text

matched_sents = []
ent_sents = defaultdict(list)
ent_count = defaultdict(int)
pattern = [{'POS': 'NOUN', 'OP': '+'},{'LOWER': 'products'}]
matcher = Matcher(nlp.vocab)
matcher.add('ProductTypes', collect_sents, pattern) # add pattern
matches = matcher(doc) # match on your text

matched_sents = []
ent_sents  = defaultdict(list)
ent_count = defaultdict(int)
pattern = [[{'POS': 'NOUN', 'OP': '+'},{'LOWER': 'fee'}]
           , [{'POS': 'NOUN', 'OP': '+'},{'LOWER': 'fees'}]]
matcher = Matcher(nlp.vocab)
matcher.add('ProductTypes', collect_sents, *pattern) # add pattern
matches = matcher(doc) # match on your text

In [99]:
matched_sents[0]

No Show policy, funds may be applied to future travel on Southwest without a change fee.  

In [100]:
ent_count

defaultdict(int,
            {'agriculture inspection fee': 1,
             'aircraft fee': 1,
             'attorneys’ fee': 3,
             'bag fee': 5,
             'baggage fee': 2,
             'change fee': 4,
             'credit card fee': 1,
             'custom fee': 1,
             'landing fee': 10,
             'passenger agriculture inspection fee': 1,
             'refund passenger fee': 1,
             'service fee': 1,
             'usage fee': 1})

In [101]:
ent_sents['aircraft fee']

[Under this rule, which was effective  December 28, 2015, the per-passenger agriculture inspection fee is $5.00 and the per-commercial  aircraft fee is $225.00.  ]

## Exercise

Extract the fee amounts

In [102]:
# TODO

### SpaCy - Text Preprocessing

In [103]:
# TODO: create this in the SpaCy notebook !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def clean_text(doc):
    ents = nlp(doc.text).ents

    # Add named entities, but only if they are a compound of more than word.
    IGNORE_ENTS = ('QUANTITY','ORDINAL','CARDINAL','DATE'
                   ,'PERCENT','MONEY','TIME')
    ents = [ent for ent in ents if 
             (ent.label_ not in IGNORE_ENTS) and (len(ent) > 2)]
    
    # add underscores to combine words in entities
    ents = [str(ent).strip().replace(' ','_') for ent in ents]
    
    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if 
           token.is_alpha and not token.is_stop]
    
    doc.extend([entity for entity in ents])
    
    return [str(term) for term in doc]

In [104]:
%%time
cleaned_text = []
for sent in matched_sents:
    text = clean_text(sent)
    cleaned_text.append(text)

CPU times: user 2.26 s, sys: 52.8 ms, total: 2.31 s
Wall time: 584 ms


In [105]:
print(cleaned_text[0])

['no', 'show', 'policy', 'fund', 'apply', 'future', 'travel', 'southwest', 'change', 'fee']


### Phrase Model

Phrase modeling is another approach to learning combinations of tokens that together represent meaningful multi-word concepts. We can develop phrase models by looping over the the words in our reviews and looking for words that co-occur (i.e., appear one after another) together much more frequently than you would expect them to by random chance. The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:

$$\frac{count(A\ B) - count_{min}}{count(A) * count(B)} * N > threshold$$

...where:

$count(A)$ is the number of times token $A$ appears in the corpus
$count(B)$ is the number of times token $B$ appears in the corpus
$count(A\ B)$ is the number of times the tokens $A\ B$ appear in the corpus in order
$N$ is the total size of the corpus vocabulary
$count_{min}$ is a user-defined parameter to ensure that accepted phrases occur a minimum number of times
$threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase
Once our phrase model has been trained on our corpus, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.

Phrase modeling is superficially similar to named entity detection in that you would expect named entities to become phrases in the model (so new york would become new_york). But you would also expect multi-word expressions that represent common concepts, but aren't specifically named entities (such as happy hour) to also become phrases in the model.

We turn to the indispensible gensim library to help us with phrase modeling — the Phrases class in particular.

SOURCE: https://github.com/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb 

In [106]:
from gensim.models.phrases import Phrases
from gensim.models.phrases import Phraser

In [107]:
# Keep only words (no numbers, no punctuation).
# Lemmatize tokens, remove punctuation

phrase_doc = []
for sent in doc.sents:
    updated_sent = []
    for token in sent:
        if token.is_alpha:
            updated_sent.append(str(token.lemma_))
    phrase_doc.append(updated_sent)

phrase_doc[0:2]

[['southwest', 'airlines', 'co'], ['annual', 'report', 'to', 'shareholder']]

In [109]:
common_terms = list(STOP_WORDS)
phrases = Phrases(phrase_doc, common_terms=common_terms, min_count=5, threshold=5)

{'indicate_by_check', 'securities_and_exchange', 'compare_with_as_the_dollar', 'million_or_percent', 'note_to_the_consolidated', 'result_of_operations', 'fuel_and_oil', 'acquisition_and_integration', 'designate_as_hedge', 'accumulated_other_comprehensive', 'chairman_of_the_board', 'qualify_for_hedge', 'balance_at_december', 'incorporate_by_reference', 'form_for_the_quarter', 'interest_be_payable', 'letter_of_credit', 'condition_and_result', 'board_of_directors', 'mark_to_market', 'discuss_in_more_detail', 'purchase_call_option', 'value_of_goodwill', 'fix_to_float', 'wanna_get_away', 'depreciation_and_amortization', 'available_for_sale', 'nos_and_incorporate', 'exhibit_and_respectively', 'number_of_share', 'incorporate_herein_by_reference', 'pursuant_to_section', 'component_of_other_gain', 'gain_or_loss', 'amend_and_restate', 'statement_of_income', 'share_of_common', 'expense_for_increase', 'meeting_of_shareholders', 'form_for_the_year', 'file_no_supplemental', 'include_as_a_component',

### Phrases Params

- **scoring:** specifies how potential phrases are scored for comparison to the threshold setting. scoring can be set with either a string that refers to a built-in scoring function, or with a function with the expected parameter names. Two built-in scoring functions are available by setting scoring to a string:

    - ‘default’: from “Efficient Estimaton of Word Representations in Vector Space” by Mikolov, et. al.: 
$$\frac{count(wordA followed by wordB) - count_{min}}{count(A) * count(B)} * N > threshold$$
    - where N is the total vocabulary size.
    - Thus, it is easier to exceed the threshold when the two words occur together often or when the two words are rare (i.e. small product)




- **common_terms:** optionnal list of “stop words” that won’t affect frequency count of expressions containing them.
    - The common_terms parameter add a way to give special treatment to common terms (aka stop words) such that their presence between two words won’t prevent bigram detection. It allows to detect expressions like “bank of america” or “eye of the beholder”.


In [155]:
bigram = Phraser(phrases)

After the one-time initialization, a Phraser will be much smaller and somewhat faster than using the full Phrases model.

Reflects the results of the source model’s min_count, threshold, and scoring settings. (You can tamper with those & create a new Phraser to try other values.)

SOURCE: https://radimrehurek.com/gensim/models/phrases.html

In [108]:
def print_phrases(phraser, text_stream, num_underscores=1):
    phrases = []
    for terms in phraser[text_stream]:
        for term in terms:
            if term.count('_') >= num_underscores:
                phrases.append(term)
    print(set(phrases))

In [154]:
print_phrases(bigram, phrase_doc, num_underscores=2)

{'indicate_by_check', 'securities_and_exchange', 'compare_with_as_the_dollar', 'million_or_percent', 'note_to_the_consolidated', 'result_of_operations', 'fuel_and_oil', 'acquisition_and_integration', 'designate_as_hedge', 'accumulated_other_comprehensive', 'chairman_of_the_board', 'qualify_for_hedge', 'balance_at_december', 'incorporate_by_reference', 'form_for_the_quarter', 'interest_be_payable', 'letter_of_credit', 'condition_and_result', 'board_of_directors', 'mark_to_market', 'discuss_in_more_detail', 'purchase_call_option', 'value_of_goodwill', 'fix_to_float', 'wanna_get_away', 'depreciation_and_amortization', 'available_for_sale', 'nos_and_incorporate', 'exhibit_and_respectively', 'number_of_share', 'incorporate_herein_by_reference', 'pursuant_to_section', 'component_of_other_gain', 'gain_or_loss', 'amend_and_restate', 'statement_of_income', 'share_of_common', 'expense_for_increase', 'meeting_of_shareholders', 'form_for_the_year', 'file_no_supplemental', 'include_as_a_component',

In [110]:
phrases = Phrases(bigram[phrase_doc], common_terms=common_terms, min_count=5,threshold=5)
trigram = Phraser(phrases)

print_phrases(trigram, bigram[phrase_doc], num_underscores=4)

{'information_require_by_this_item', 'compare_with_as_the_dollar', 'reference_to_exhibit_and_respectively', 'prepaid_expense_and_other_current_asset', 'management_discussion_and_analysis_of_financial', 'chief_executive_officer_and_chief', 'set_forth_under_the_head', 'goodwill_and_indefinite_live_intangible', 'reference_to_exhibit_to_the_company_annual', 'agreements_nos_and_incorporate_by_reference', 'report_on_form_for_the_year_end', 'form_for_the_year_end_december', 'fiscal_year_and_interim_period', 'increase_be_more_than_offset', 'fair_value_of_fuel_derivative', 'condition_and_result_of_operations', 'annual_report_on_form_for_the_year', 'component_of_other_gain_loss_net', 'company_quarterly_report_on_form', 'note_regard_use_of_non', 'company_annual_report_on_form', 'exhibit_to_the_company_annual_report', 'form_for_the_quarter_end_june', 'quarterly_report_on_form_for_the_quarter', 'report_on_form_for_the_quarter_end', 'file_no_supplemental_agreement_no_incorporate', 'point_sell_to_bus

In [111]:
for doc_num in [6,29]:
    print('DOC NUMBER: {}'.format(doc_num))
    print('ORIGINAL SENTENT: {}\n'.format(phrase_doc[doc_num]))
    print('BIGRAM: {}\n'.format(bigram[phrase_doc[doc_num]]))
    print('TRIGRAM: {}'.format(trigram[bigram[phrase_doc[doc_num]]]))
    print('\n')

DOC NUMBER: 6
ORIGINAL SENTENT: ['-PRON-', 'result', 'be', 'furth', 'boost', 'by', 'the', 'significant', 'reduction', 'in', 'energy', 'price', 'generally', 'range', 'bind', 'in', 'the', 'a', 'barrel', 'range', 'for', 'brent', 'crude', 'oil']

BIGRAM: ['-PRON-', 'result', 'be', 'furth', 'boost', 'by', 'the', 'significant', 'reduction', 'in', 'energy_price', 'generally', 'range', 'bind', 'in', 'the', 'a', 'barrel', 'range', 'for', 'brent_crude', 'oil']

TRIGRAM: ['-PRON-', 'result', 'be', 'furth', 'boost', 'by', 'the', 'significant', 'reduction', 'in', 'energy_price', 'generally', 'range', 'bind', 'in', 'the', 'a', 'barrel', 'range', 'for', 'brent_crude_oil']


DOC NUMBER: 29
ORIGINAL SENTENT: ['operate', 'expense', 'per', 'available', 'seat', 'mile', 'casm', 'increase', 'percent', 'year', 'over', 'year']

BIGRAM: ['operate', 'expense', 'per', 'available_seat', 'mile', 'casm', 'increase_percent', 'year_over_year']

TRIGRAM: ['operate', 'expense', 'per', 'available_seat_mile', 'casm', 'in

### Review Phrase Model

In [112]:
def clean_text(doc):
    ents = nlp(doc.text).ents

    # Add named entities, but only if they are a compound of more than word.
    IGNORE_ENTS = ('QUANTITY','ORDINAL','CARDINAL','DATE'
                   ,'PERCENT','MONEY','TIME')
    ents = [ent for ent in ents if 
             (ent.label_ not in IGNORE_ENTS) and (len(ent) > 2)]
    
    # add underscores to combine words in entities
    ents = [str(ent).strip().replace(' ','_') for ent in ents]
    
    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if 
           token.is_alpha and not token.is_stop]
    
    doc.extend([entity for entity in ents])
    
    return [str(term) for term in doc]

    
# combined terms before phrase_model (entities and/or noun chunks)
before_phrase = []
for sent in matched_sents:
    text = clean_text(sent)
    for term in text:
        if '_' in term:
            before_phrase.append(term)

print(set(before_phrase))

{'Plant_Health_Inspection_Service', 'Section_2_of_the_Sherman_Act', '13.9__Acquisition', 'the_Sherman_Act', 'the_U.S._Department', '938__Acquisition', 'Section_1', 'Consolidated_Amended_Complaint', 'The_Passenger_Protection_Rules'}


In [113]:
def clean_text(doc):
    ents = nlp(doc.text).ents

    # Add named entities, but only if they are a compound of more than word.
    IGNORE_ENTS = ('QUANTITY','ORDINAL','CARDINAL','DATE'
                   ,'PERCENT','MONEY','TIME')
    ents = [ent for ent in ents if 
             (ent.label_ not in IGNORE_ENTS) and (len(ent) > 2)]
    
    # add underscores to combine words in entities
    ents = [str(ent).strip().replace(' ','_') for ent in ents]
    
    # clean text for phrase model
    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc_ = [token.lemma_ for token in doc if token.is_alpha]
    phrase_text = [str(term) for term in doc_]
    sent = trigram[bigram[phrase_text]]
    phrases = []
    for term in sent:
        if '_' in term:
            phrases.append(term)

    # remove stops words - 
    # separate step as they are needed for the phrase model
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # add phrases and entities
    doc.extend([entity for entity in ents])
    clean_text = [str(term) for term in doc] + phrases

    return clean_text

# combined terms after phrase model
after_phrase = []
for sent in matched_sents:
    text = clean_text(sent)
    for term in text:
        if '_' in term:
            after_phrase.append(term)

print(set(after_phrase))

{'compare_with_as_the_dollar', 'Section_2_of_the_Sherman_Act', 'acquisition_and_integration', 'operating_lease', 'flight_crew', 'flight_date', 'operating_expense', 'customer_service', 'dollar_basis', 'passenger_revenue', 'increase_be_more_than_offset', 'depreciation_and_amortization', 'the_U.S._Department', '938__Acquisition', 'aircraft_rental', 'air_travel', 'check_bag', 'passenger_protection', 'consolidated_statement_of_income', 'Plant_Health_Inspection_Service', 'total_operating', 'low_fare', 'straight_line', 'operating_income', 'Consolidated_Amended_Complaint', 'decrease_percent', 'company_currently_expect', 'future_travel', 'bag_fee', 'landing_fee', 'rental_expense', '13.9__Acquisition', 'the_Sherman_Act', 'percent_increase_in_capacity', 'expense_associate', 'credit_card', 'Section_1', 'asm_basis', 'The_Passenger_Protection_Rules', 'increase_by_million_or_percent'}


In [114]:
cleaned_text = []
for sent in matched_sents:
    text = clean_text(sent)
    cleaned_text.append(text)

print(cleaned_text[0:2])

[['no', 'show', 'policy', 'fund', 'apply', 'future', 'travel', 'southwest', 'change', 'fee', 'future_travel'], ['for', 'example', 'company', 'transfarencysm', 'campaign', 'emphasize', 'southwest', 'approach', 'treat', 'customer', 'fairly', 'honestly', 'respectfully', 'low', 'fare', 'unexpected', 'bag', 'fee', 'change', 'fee', 'hidden', 'fee', 'low_fare', 'bag_fee']]


# Text Vectorization (gensim)

In [116]:
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
import pandas as pd
import scipy.sparse

In [118]:
texts = cleaned_text

text = texts[0]

In [119]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

In [120]:
dictionary.filter_extremes(no_below=2, no_above=0.4)

Here we assigned a unique integer ID to all words appearing in the processed corpus with the gensim.corpora.dictionary.Dictionary class. This sweeps across the texts, collecting word counts and relevant statistics. In the end, we see there are twelve distinct words in the processed corpus, which means each document will be represented by twelve numbers (ie., by a 12-D vector). To see the mapping between words and their ids:

In [121]:
print(dictionary.token2id)

{'show': 0, 'apply': 1, 'travel': 2, 'southwest': 3, 'change': 4, 'for': 5, 'example': 6, 'company': 7, 'transfarencysm': 8, 'campaign': 9, 'emphasize': 10, 'approach': 11, 'treat': 12, 'customer': 13, 'fairly': 14, 'honestly': 15, 'respectfully': 16, 'low': 17, 'fare': 18, 'unexpected': 19, 'bag': 20, 'hidden': 21, 'low_fare': 22, 'bag_fee': 23, 'the': 24, 'service': 25, 'charge': 26, 'pay': 27, 'airfare': 28, 'passenger': 29, 'protection': 30, 'rules': 31, 'require': 32, 'airline': 33, 'compensation': 34, 'deny': 35, 'boarding': 36, 'involuntarily': 37, 'oversold': 38, 'flight': 39, 'ii': 40, 'refund': 41, 'check': 42, 'permanently': 43, 'lose': 44, 'luggage': 45, 'iii': 46, 'prominently': 47, 'disclose': 48, 'potential': 49, 'optional': 50, 'ancillary': 51, 'website': 52, 'iv': 53, 'cancel': 54, 'oversell': 55, 'unable': 56, 'advantage': 57, 'The_Passenger_Protection_Rules': 58, 'passenger_protection': 59, 'check_bag': 60, 'include': 61, 'increase': 62, 'baggage': 63, 'in': 64, 'dec

In [122]:
print(text)

dictionary.doc2bow(text)

['no', 'show', 'policy', 'fund', 'apply', 'future', 'travel', 'southwest', 'change', 'fee', 'future_travel']


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

The function doc2bow() simply counts the number of occurrences of each distinct word, converts the word to its integer word id and returns the result as a bag-of-words--a sparse vector, in the form of [(word_id, word_count), ...].

As the token_id is 0 for "human" and 2 for "computer", the new document “Human computer interaction” will be transformed to [(0, 1), (2, 1)]. The words "computer" and "human" exist in the dictionary and appear once. Thus, they become (0, 1), (2, 1) respectively in the sparse vector. The word "interaction" doesn't exist in the dictionary and, thus, will not show up in the sparse vector. The other ten dictionary words, that appear (implicitly) zero times, will not show up in the sparse vector and , ,there will never be a element in the sparse vector like (3, 0).

For people familiar with scikit learn, doc2bow() has similar behaviors as calling transform() on CountVectorizer. doc2bow() can behave like fit_transform() as well. For more details, please look at gensim API Doc.

In [123]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(sent) for sent in texts]

We used our old corpus from tutorial 1 to initialize (train) the transformation model. Different transformations may require different initialization parameters; in case of TfIdf, the “training” consists simply of going through the supplied corpus once and computing document frequencies of all its features. Training other models, such as Latent Semantic Analysis or Latent Dirichlet Allocation, is much more involved and, consequently, takes much more time.

In [124]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model
tfidf

In [127]:
tfidf_corpus = []
tfidf_transform = tfidf[corpus]
for doc in tfidf_transform:
    tfidf_corpus.append(doc)

tfidf_corpus[0]

[(0, 0.537680364152946),
 (1, 0.537680364152946),
 (2, 0.40326027311470947),
 (3, 0.35998666929218426),
 (4, 0.35998666929218426)]

# Topic Models (Gensim)

### Latent Semantic Indexing (LSI)
Latent semantic indexing (LSI) is an indexing and retrieval method that uses a mathematical technique called singular value decomposition (SVD) to identify patterns in the relationships between the terms and concepts contained in an unstructured collection of text. LSI is based on the principle that words that are used in the same contexts tend to have similar meanings. A key feature of LSI is its ability to extract the conceptual content of a body of text by establishing associations between those terms that occur in similar contexts.[18]

The method, also called latent semantic analysis (LSA), uncovers the underlying latent semantic structure in the usage of words in a body of text and how it can be used to extract the meaning of the text in response to user queries, commonly referred to as concept searches. Queries, or concept searches, against a set of documents that have undergone LSI will return results that are conceptually similar in meaning to the search criteria even if the results don’t share a specific word or words with the search criteria.

### Overview
LSA can use a term-document matrix which describes the occurrences of terms in documents; it is a sparse matrix whose rows correspond to terms and whose columns correspond to documents. A typical example of the weighting of the elements of the matrix is tf-idf (term frequency–inverse document frequency): the weight of an element of the matrix is proportional to the number of times the terms appear in each document, where rare terms are upweighted to reflect their relative importance.

This matrix is also common to standard semantic models, though it is not necessarily explicitly expressed as a matrix, since the mathematical properties of matrices are not always used.

### Rank Lowering
After the construction of the occurrence matrix, LSA finds a low-rank approximation[4] to the term-document matrix. There could be various reasons for these approximations:

The original term-document matrix is presumed too large for the computing resources; in this case, the approximated low rank matrix is interpreted as an approximation (a "least and necessary evil").
The original term-document matrix is presumed noisy: for example, anecdotal instances of terms are to be eliminated. From this point of view, the approximated matrix is interpreted as a de-noisified matrix (a better matrix than the original).
The original term-document matrix is presumed overly sparse relative to the "true" term-document matrix. That is, the original matrix lists only the words actually in each document, whereas we might be interested in all words related to each document—generally a much larger set due to synonymy.
The consequence of the rank lowering is that some dimensions are combined and depend on more than one term:

{(car), (truck), (flower)} --> {(1.3452 * car + 0.2828 * truck), (flower)}

This mitigates the problem of identifying synonymy, as the rank lowering is expected to merge the dimensions associated with terms that have similar meanings. It also mitigates the problem with polysemy, since components of polysemous words that point in the "right" direction are added to the components of words that share a similar meaning. Conversely, components that point in other directions tend to either simply cancel out, or, at worst, to be smaller than components in the directions corresponding to the intended sense.


### Use Cases
- Compare the documents in the low-dimensional space (data clustering, document classification).
- Find similar documents across languages, after analyzing a base set of translated documents (cross language retrieval).
- Find relations between terms (synonymy and polysemy).
- Given a query of terms, translate it into the low-dimensional space, and find matching documents (information retrieval).
- Find the best similarity between small groups of terms, in a semantic way (i.e. in a context of a knowledge corpus), as for example in multi choice questions MCQ answering model.
- Expand the feature space of machine learning / text mining systems 
- Analyze word association in text corpus 

Synonymy and polysemy are fundamental problems in natural language processing:
* Synonymy is the phenomenon where different words describe the same idea. Thus, a query in a search engine may fail to retrieve a relevant document that does not contain the words which appeared in the query. For example, a search for "doctors" may not return a document containing the word "physicians", even though the words have the same meaning.

* Polysemy is the phenomenon where the same word has multiple meanings. So a search may retrieve irrelevant documents containing the desired words in the wrong meaning. For example, a botanist and a computer scientist looking for the word "tree" probably desire different sets of documents.

### Limitations
LSA cannot capture polysemy (i.e., multiple meanings of a word) because each occurrence of a word is treated as having the same meaning due to the word being represented as a single point in space.[citation needed]. For example, the occurrence of "chair" in a document containing "The Chair of the Board" and in a separate document containing "the chair maker" are considered the same. The behavior results in the vector representation being an average of all the word's different meanings in the corpus, which can make it difficult for comparison. However, the effect is often lessened due to words having a predominant sense throughout a corpus (i.e. not all meanings are equally likely).

SOURCE: https://en.wikipedia.org/wiki/Latent_semantic_analysis#cite_note-38

In [128]:
# initialize an LSI transformation
lsi = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=2) 

In [129]:
# create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
corpus_lsi = lsi[tfidf_corpus] 

In [130]:
lsi.print_topics(2, num_words=5)

[(0,
  '-0.285*"increase" + -0.283*"percent" + -0.276*"rental" + -0.248*"rental_expense" + -0.236*"expense"'),
 (1,
  '0.304*"passenger" + 0.237*"service" + 0.162*"pay" + -0.161*"percent" + 0.148*"damage"')]

### LDA

In [131]:
ldamodel = LdaModel(
        tfidf_corpus, num_topics=2, id2word=dictionary, passes=200)

ldamodel.print_topics(num_topics=2, num_words=5)

[(0,
  '0.023*"passenger" + 0.020*"service" + 0.018*"southwest" + 0.017*"change" + 0.016*"inspection"'),
 (1,
  '0.019*"rental" + 0.017*"increase" + 0.017*"expense" + 0.016*"landing" + 0.016*"landing_fee"')]

In [132]:
ldamodel.top_topics(corpus)

[([(0.018974580317509881, 'rental'),
   (0.017328061230754692, 'increase'),
   (0.016996578265364378, 'expense'),
   (0.01611641963098397, 'landing'),
   (0.01611641963097742, 'landing_fee'),
   (0.015348498895761549, 'percent'),
   (0.014633059876339836, 'rental_expense'),
   (0.01435251539191755, 'damage'),
   (0.013658045675954951, 'cost'),
   (0.013198770476051174, 'in'),
   (0.013004984725550211, 'airtran'),
   (0.012344643086849148, 'compare'),
   (0.012303525281150068, 'seek'),
   (0.012303525281149821, 'relief'),
   (0.01230352528114676, 'injunctive'),
   (0.01193330686936139, 'on'),
   (0.011436652843529512, 'court'),
   (0.011368825590748468, 'asm'),
   (0.01128345481733793, 'delta'),
   (0.011161364311113274, 'increase_by_million_or_percent')],
  -13.726536223002389),
 ([(0.023297363064785311, 'passenger'),
   (0.02030638240568073, 'service'),
   (0.017851455879786128, 'southwest'),
   (0.017125052016583774, 'change'),
   (0.016391845591608657, 'inspection'),
   (0.016391845

## Visdom

Install it with:

`conda install visdom`

Start the server:

`python -m visdom.server`

Visdom now can be accessed at http://localhost:8097 in the browser.


# LDA Training Visualization

Knowing about the progress and performance of a model, as we train them, could be very helpful in understanding it’s learning process and makes it easier to debug and optimize them. In this notebook, we will learn how to visualize training statistics for LDA topic model in gensim. To monitor the training, a list of Metrics is passed to the LDA function call for plotting their values live as the training progresses. 


<img src="../raw_data/images/visdom_graph.png">


SOURCE: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Training_visualizations.ipynb
Visdom SOURCE: https://github.com/facebookresearch/visdom

In [149]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback for hold_out and test corpus
#pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="visdom", title="Perplexity (hold_out)")
#pl_test = PerplexityMetric(corpus=test_corpus, logger="visdom", title="Perplexity (test)")


# u_mass, c_v, c_uci, c_npmi.
# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=tfidf_corpus, coherence="c_v", logger="visdom", title="Coherence (u_mass)", window_size=110)
#ch_cv = CoherenceMetric(corpus=tfidf_corpus, texts=texts, coherence="c_v", logger="visdom", title="Coherence (c_v)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", title="Diff (kullback_leibler)")
convergence_kl = ConvergenceMetric(distance="jaccard", logger="visdom", title="Convergence (jaccard)")

callbacks = [#pl_holdout, pl_test,  ch_cv,
    diff_kl, convergence_kl, ch_umass]

# training LDA model
ldamodel = LdaModel(
        tfidf_corpus, num_topics=20, id2word=dictionary, passes=200,  callbacks=callbacks)

#model = ldamodel.LdaModel(corpus=training_corpus, id2word=dictionary, num_topics=35, passes=50, chunksize=1500, iterations=200, alpha='auto', callbacks=callbacks)

ValueError: ('%s coherence is not currently supported.', 10)

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def topic_model(text):

    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(text)
    
    dictionary.filter_extremes(no_below=2, no_above=0.4)

    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(sent) for sent in text] # for text in texts]

    tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

    tfidf_corpus = []
    tfidf_transform = tfidf[corpus]
    for doc in tfidf_transform:
        tfidf_corpus.append(doc)

    ldamodel = LdaModel(
            tfidf_corpus, num_topics=2, id2word = dictionary, passes=20)

    print(ldamodel.top_topics(corpus))

    for topic in ldamodel.print_topics(num_topics=2, num_words=5):
#        print(topic)
        print()

topic_model(cleaned_text)

def put_text_into_topics():
    data = []
    for ind, topics in enumerate(ldamodel.get_document_topics(corpus)):
        d = {'text':' '.join(text[ind])}
        for topic in topics:
            d[topic[0]] = topic[1]
        data.append(d)
    
    df = pd.DataFrame(data)
    return df
    #df.to_csv(r'dlete.csv', index=False)